<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 10


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

Создать базовый класс Invoice в C#, который будет представлять информацию о
фактурах за поставленные товары или оказанные услуги. На основе этого класса
разработать 2-3 производных класса, демонстрирующих принципы наследования и
полиморфизма. В каждом из классов должны быть реализованы новые атрибуты и
методы, а также переопределены некоторые методы базового класса для
демонстрации полиморфизма.

Требования к базовому классу Invoice:

• Атрибуты: Номер фактуры (InvoiceNumber), Дата выдачи (IssueDate), Общая
сумма (TotalAmount).

• Методы:
1. CalculateTotal(): метод для расчета общей суммы по фактуре.
2. AddLine(LineItem lineItem): метод для добавления позиции в фактуру.
3. RemoveLine(LineItem lineItem): метод для удаления позиции из
фактуры.

Требования к производным классам:
1. ТоварнаяФактура (GoodsInvoice): Должна содержать дополнительные
атрибуты, такие как Дата поставки (SupplyDate). Метод AddLine() должен
быть переопределен для добавления информации о дате поставки товара
при добавлении позиции.
2. УслуговаяФактура (ServiceInvoice): Должна содержать дополнительные
атрибуты, такие как Дата оказания услуги (ServiceDate).
Метод RemoveLine() должен быть переопределен для добавления
информации о причине аннулирования услуги при удалении позиции.
3. КомбинированнаяФактура (CombinedInvoice) (если требуется третий класс):
Должна содержать дополнительные атрибуты, такие как Наличие возврата
(ReturnAllowed). Метод CalculateTotal() должен быть переопределен для
учета возможного возврата товара или услуги при расчете общей суммы.

#### Дополнительное задание
Добавьте к сущестующим классам конструктора классов с использованием гетторов и сетторов и реализуйте взаимодействие объектов между собой

<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [1]:
using System;
using System.Collections.Generic;

class LineItem
{
    private string _description;
    private decimal _price;
    private int _quantity;
    private bool _returnAllowed;
    private int _returnedQuantity;

    public string Description
    {
        get { return _description; }
        set { _description = value; }
    }

    public decimal Price
    {
        get { return _price; }
        set { _price = value; }
    }

    public int Quantity
    {
        get { return _quantity; }
        set { _quantity = value; }
    }

    public bool ReturnAllowed
    {
        get { return _returnAllowed; }
        set { _returnAllowed = value; }
    }

    public int ReturnedQuantity
    {
        get { return _returnedQuantity; }
        set { _returnedQuantity = value; }
    }

    public LineItem(string description, decimal price, int quantity, bool returnAllowed)
    {
        _description = description;
        _price = price;
        _quantity = quantity;
        _returnAllowed = returnAllowed;
        _returnedQuantity = 0;
    }

    public decimal GetTotal()
    {
        return _price * (_quantity - _returnedQuantity);
    }

    public virtual bool Return(int returnQuantity)
    {
        if (_returnAllowed && returnQuantity <= (_quantity - _returnedQuantity))
        {
            _returnedQuantity += returnQuantity;
            return true;
        }
        return false;
    }
}

abstract class Invoice
{
    private string _invoiceNumber;
    private DateTime _issueDate;
    private decimal _totalAmount;
    protected List<LineItem> lineItems;

    public string InvoiceNumber
    {
        get { return _invoiceNumber; }
        set { _invoiceNumber = value; }
    }

    public DateTime IssueDate
    {
        get { return _issueDate; }
        set { _issueDate = value; }
    }

    public decimal TotalAmount
    {
        get { return _totalAmount; }
        protected set { _totalAmount = value; }
    }

    public Invoice(string invoiceNumber, DateTime issueDate)
    {
        _invoiceNumber = invoiceNumber;
        _issueDate = issueDate;
        lineItems = new List<LineItem>();
    }

    public virtual void CalculateTotal()
    {
        _totalAmount = 0;
        foreach (var item in lineItems)
        {
            _totalAmount += item.GetTotal();
        }
    }

    public virtual void AddLine(LineItem lineItem)
    {
        lineItems.Add(lineItem);
    }

    public virtual void RemoveLine(LineItem lineItem)
    {
        lineItems.Remove(lineItem);
        CalculateTotal();
        Console.WriteLine($"Удалена позиция: {lineItem.Description}\nОбщая сумма по комбинированной фактуре после удаления: {_totalAmount}");
    }
}

class GoodsItem : LineItem
{
    private DateTime _supplyDate;

    public DateTime SupplyDate
    {
        get { return _supplyDate; }
        set { _supplyDate = value; }
    }

    public GoodsItem(string description, decimal price, int quantity, DateTime supplyDate, bool returnAllowed)
        : base(description, price, quantity, returnAllowed)
    {
        _supplyDate = supplyDate;
    }
}

class ServiceItem : LineItem
{
    private DateTime _serviceDate;

    public DateTime ServiceDate
    {
        get { return _serviceDate; }
        set { _serviceDate = value; }
    }

    public ServiceItem(string description, decimal price, int quantity, DateTime serviceDate, bool returnAllowed)
        : base(description, price, quantity, returnAllowed)
    {
        _serviceDate = serviceDate;
    }
}

class CombinedInvoice : Invoice
{
    private bool _returnAllowed;
    private int _totalReturnedItems;

    public bool ReturnAllowed
    {
        get { return _returnAllowed; }
        set { _returnAllowed = value; }
    }

    public int TotalReturnedItems
    {
        get { return _totalReturnedItems; }
        private set { _totalReturnedItems = value; }
    }

    public CombinedInvoice(string invoiceNumber, DateTime issueDate, bool returnAllowed)
        : base(invoiceNumber, issueDate)
    {
        _returnAllowed = returnAllowed;
    }

    public override void AddLine(LineItem lineItem)
    {
        base.AddLine(lineItem);

        if (lineItem is GoodsItem goodsItem && lineItem.ReturnAllowed)
        {
            Console.WriteLine($"Добавлен товар: {goodsItem.Description}, количество: {goodsItem.Quantity}. Дата поставки товара: {goodsItem.SupplyDate:dd.MM.yyyy}. Данный товар можно вернуть.");
        }
        else if (lineItem is GoodsItem goodsItemNonReturn && !lineItem.ReturnAllowed)
        {
            Console.WriteLine($"Добавлен товар: {goodsItemNonReturn.Description}, количество: {goodsItemNonReturn.Quantity}. Дата поставки товара: {goodsItemNonReturn.SupplyDate:dd.MM.yyyy}. Данный товар нельзя вернуть.");
        }
        else if (lineItem is ServiceItem serviceItem && lineItem.ReturnAllowed)
        {
            Console.WriteLine($"Добавлена услуга: {serviceItem.Description}, количество: {serviceItem.Quantity}. Дата оказании услуги: {serviceItem.ServiceDate:dd.MM.yyyy}. Данную услугу можно вернуть.");
        }
        else if (lineItem is ServiceItem serviceItemNonReturn && !lineItem.ReturnAllowed)
        {
            Console.WriteLine($"Добавлена услуга: {serviceItemNonReturn.Description}, количество: {serviceItemNonReturn.Quantity}. Дата оказании услуги: {serviceItemNonReturn.ServiceDate:dd.MM.yyyy}. Данную услугу нельзя вернуть.");
        }

        CalculateTotal();
    }

    public void ReturnItem(LineItem lineItem, int returnQuantity)
    {
        if (!_returnAllowed)
        {
            Console.WriteLine("Возврат товара или услуги для данной фактуры запрещен.");
            return;
        }

        if (!lineItem.ReturnAllowed)
        {
            Console.WriteLine($"Возврат для товара/услуги {lineItem.Description} запрещен.");
            return;
        }

        if (lineItem.Return(returnQuantity))
        {
            Console.WriteLine($"Возвращено количество: {returnQuantity} шт. из позиции: {lineItem.Description}. Остаток позиции в фактуре: {lineItem.Quantity - lineItem.ReturnedQuantity}.");
            _totalReturnedItems += returnQuantity;
            CalculateTotal();
            Console.WriteLine($"Общая сумма по комбинированной фактуре после возврата: {TotalAmount}.");
        }
        else
        {
            Console.WriteLine($"Невозможно вернуть {returnQuantity} шт. из {lineItem.Description}. Возможно, запрашиваемое количество больше, чем доступно для возврата или же {lineItem.Description} не находится в фактуре.");
        }

        CalculateTotal();
    }
}

// Пример использования
CombinedInvoice combinedInvoice = new CombinedInvoice("INV003", DateTime.Now, true);

GoodsItem item1 = new GoodsItem("Товар 1", 100, 25, DateTime.Now.AddDays(7), true);
GoodsItem item2 = new GoodsItem("Товар 2", 200, 3, DateTime.Now.AddDays(10), false);

combinedInvoice.AddLine(item1);
combinedInvoice.AddLine(item2);

ServiceItem service1 = new ServiceItem("Услуга 1", 300, 0, DateTime.Now.AddDays(-2), true);
ServiceItem service2 = new ServiceItem("Услуга 2", 400, 1, DateTime.Now.AddDays(1), false);

combinedInvoice.AddLine(service1);
combinedInvoice.AddLine(service2);

combinedInvoice.CalculateTotal();
Console.WriteLine($"Общая сумма фактуры: {combinedInvoice.TotalAmount}");

combinedInvoice.ReturnItem(item1, 1);

Console.WriteLine($"Общее количество возвращенных товаров и услуг: {combinedInvoice.TotalReturnedItems}.");
